### 🧠 IA + Oracle: Hola mundo agentic
Este notebook guía a los alumnos en una práctica básica para conectar con una base de datos Oracle, generar consultas SQL desde lenguaje natural usando distintos agentes vía [AISuite](https://github.com/andrewyng/aisuite/), y ejecutar esas consultas de forma segura.

El objetivo es que a partir de una consulta en lenguaje natural, un primer agente genere el código sql asociado a esa consulta del usuario. Después se consultará la base de datos y se obtendrá el resultado en formato tabular que se mostrará al usuario. Esa información se enviará a otro agente que interpretará el resultado y se lo explicará al usuario. 

En el fichero Readme.md se explica la configuración necesaria para que este jupiter notebook funcione correctamente.

### 🔄 Flujo del Sistema Agéntico

![Diagrama del Sistema Agéntico](images/diagramaagentic1-notrans.png)

Este diagrama muestra el flujo completo del sistema de agentes que conecta la inteligencia artificial con la Base de Datos Oracle para procesar consultas en lenguaje natural.

El primer paso será cargar las variables de entorno que están guardadas en el fichero .env donde se debe guardar la información de:
- Conexión a la base de datos Oracle ORACLE_HOST, ORACLE_PORT, ORACLE_SID, ORACLE_USER, ORACLE_PASSWORD
- Dependiendo de la IA que se vaya a utilizar se necesitará:
    - OpenAI- OPENAI_API_KEY
    - Anthropic - ANTHROPIC_API_KEY
    - Google - GOOGLE_PROJECT_ID, GOOGLE_REGION, GOOGLE_APPLICATION_CREDENTIALS


In [ ]:
# 🔐 Cargar variables de entorno
from dotenv import load_dotenv
import os

load_dotenv()
# muestra las variables de entorno cargadas (opcional)
# for key, value in os.environ.items():
#     if key in ["ORACLE_HOST", "ORACLE_PORT", "ORACLE_SID", "ORACLE_USER", "ORACLE_PASSWORD", "GOOGLE_PROJECT_ID", "GOOGLE_REGION", "GOOGLE_APPLICATION_CREDENTIALS", "OPENAI_API_KEY"]:
#         print(f"{key}: {value}")
print("Variables de entorno cargadas.")        

ORACLE_HOST = os.getenv("ORACLE_HOST")
ORACLE_PORT = os.getenv("ORACLE_PORT")
ORACLE_SID = os.getenv("ORACLE_SID")
ORACLE_USER = os.getenv("ORACLE_USER")
ORACLE_PASSWORD = os.getenv("ORACLE_PASSWORD")
# GOOGLE API KEY no funciona con AISuite. Solo funciona con Google Vertex AI
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# GOOGLE_PROJECT_ID, GOOGLE_REGION, GOOGLE_APPLICATION_CREDENTIALS
GOOGLE_PROJECT_ID = os.getenv("GOOGLE_PROJECT_ID")
GOOGLE_REGION = os.getenv("GOOGLE_REGION")
GOOGLE_APPLICATION_CREDENTIALS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

In [ ]:
# 🔗 Conexión a Oracle
import oracledb

# Creación de la cadena de conexión (DSN)
dsn = oracledb.makedsn(ORACLE_HOST, ORACLE_PORT, sid=ORACLE_SID)
# Conexión a la base de datos Oracle
connection = oracledb.connect(
    user=ORACLE_USER,
    password=ORACLE_PASSWORD,
    dsn=dsn
)
# Verificar la conexión ejecutando una consulta simple
cursor = connection.cursor()
cursor.execute("SELECT * FROM Dual")
for row in cursor:
    print(row)

La estructura de base de datos de ejemplo es muy sencilla. Se muestra el diagrama entidad relación.
![Modelo ER](images/ERBDPEDIDOS.png)

In [ ]:
# 🧠 Entrada en lenguaje natural. Algunos ejemplos para la base de datos
# de clientes productos y pedidos.
pregunta = "¿Cuántos pedidos se hicieron en octubre?"
pregunta = "Selecciona todos los pedidos"
pregunta = "Cuantos  y qué pedidos hizo Ana Gamez"
pregunta = "Quiero saber los productos que ha comprado Ana Gamez y lo que le costó cada uno"



Es importante pasarle a los agentes la estructura de la base de datos. Para que las consultas que generen tengan sentido. Eso se hace en el siguiente bloque usando un sistema de descubrimiento.

In [ ]:
# 🗂️ Obtener esquema de la base de datos
# genera Consulta para obtener tablas y columnas del usuario actual debes seleccionar el 
#usuario actual
cursor.execute(f"""
    SELECT table_name, column_name
    FROM all_tab_columns
    WHERE owner = UPPER('{ORACLE_USER}')
    ORDER BY table_name, column_id
""")

esquema = {}
for table, column in cursor:
    esquema.setdefault(table, []).append(column)
print("Esquema de la base de datos:")
print(esquema)

# Alternativa más simple pero menos profesional es indicar el esquema a mano
#esquema = """
#Tablas disponibles:
#- clientes(id_cliente, nombre, email, ciudad)
#- productos(id_producto, nombre, precio, categoria)
#- pedidos(id_pedido, id_cliente, id_producto, fecha_pedido, cantidad)
#"""




In [ ]:
# 🤖 Generación de SQL con Gemini|OpenAI|Anthropic vía AISuite
import aisuite as ai

client = ai.Client()


messages = [
    {"role": "system", "content": f"Eres un asistente experto en SQL para Oracle. Genera solo la consulta SQL compatible con Oracle.Usa este esquema:\n{esquema}"},
    {"role": "user", "content": pregunta}
   
]


#Seleccionar el modelo que se desee usar. 
response = client.chat.completions.create(
    #model="openai:gpt-3.5-turbo",  # o "anthropic:claude-2" si usas Anthropic
    model="google:gemini-2.0-flash-001",
    messages=messages
)


# Extrae y muestra el SQL (eliminando el formato Markdown que aparece en openAI)
raw_sql = response.choices[0].message.content.strip()
if raw_sql.startswith("```sql"):
# Elimina la primera línea (```sql) y la última (```)
    sql_generado = "\n".join(raw_sql.split("\n")[1:-1]).strip()
else:
    sql_generado = raw_sql
print("Consulta generada por IA:")
sql_generado = sql_generado.replace(";", "")  # Elimina el punto y coma si está presente
#eliminar saltos de línea y tabulaciones pues dan error en Oracle
sql_generado = sql_generado.replace("\n", " ").replace("\t", " ")
print(sql_generado)

In [ ]:
# 🧪 Ejecución del SQL
try:
    #sql_generado = "Select * from pedidos" # para probar directamente
    cursor.execute(sql_generado)
    resultados = cursor.fetchall()
    print("Resultados de la consulta:")
    for fila in resultados:
        print(fila)
except Exception as e:
    print("Error al ejecutar la consulta:", e)

In [ ]:
# 📊 Visualización en formato tabular
import pandas as pd

columnas = [col[0] for col in cursor.description]
df = pd.DataFrame(resultados, columns=columnas)
#para pasarselo a la IA y que lo interprete en el siguiente paso
texto_resultado = df.to_markdown(index=False)
df.head()

Aqui se crea el segundo Agente que puede ser con el mismo o diferente proveedor y será el encargado de coger el resultado de la consulta y analizarlo y explicarselo al usuario.

In [ ]:
messages = [
    {"role": "system", "content": f"Eres un experto en análisis de datos. Resume e interpreta los resultados de una consulta SQL realizada sobre este esquema: {esquema}"},
    {"role": "user", "content": f"La consulta fue: {pregunta}\n\nY los resultados fueron:\n\n{texto_resultado}"}
]

response = client.chat.completions.create(
    model="openai:gpt-4o",  # o el modelo que prefieras
    messages=messages
)

respuesta_final = response.choices[0].message.content.strip()
print("🧠 Respuesta interpretada por IA:")
print(respuesta_final)

### 🧠 Reflexión
- ¿Qué otras preguntas podrías hacer?
- ¿Cómo podrías validar que el SQL generado es seguro?
- ¿Qué diferencias hay entre usar Gemini y otros modelos?

### 🔄 Extensión
- Cambia el modelo a `openai:gpt-4o` o `anthropic:claude-3-sonnet`
- Añade un agente que valide que la consulta generada por el primero es correcta.
- Haz que el agente planifique varios pasos (consulta + visualización)